In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout, Input
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from scipy.optimize import fmin_l_bfgs_b

from keras.applications import vgg19
from keras import utils as np_utils
from keras import backend as K

import numpy as np
import tensorflow as tf

Using TensorFlow backend.


In [186]:
#uploading image 
from PIL import Image
img= Image.open("Desktop/images.jpeg")
orignal_size= img.size
print(orignal_size)
img_width= orignal_size[0]
img_height=orignal_size[1]
image_size= (img_height, img_width, 3)
print(image_size)
#img = img.resize((299,299))
#img.show()
x= img_to_array(img)
x = np.expand_dims(x, axis=0)
x=x[:,:,:,::-1]


(275, 183)
(183, 275, 3)


In [204]:
x = np.random.random((1, img_height, img_width, 3)) * 20 + 128.
x= x.reshape((img_height, img_width, 3))
temp= array_to_img(x)
temp.show()

In [187]:
#Downloading VGGNET
dream = Input(batch_shape=(1,)+image_size)
model = vgg19.VGG19(input_tensor=dream, weights="imagenet", include_top=False)


In [188]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (1, 183, 275, 3)          0         
_________________________________________________________________
block1_conv1 (Conv2D)        (1, 183, 275, 64)         1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (1, 183, 275, 64)         36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (1, 91, 137, 64)          0         
_________________________________________________________________
block2_conv1 (Conv2D)        (1, 91, 137, 128)         73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (1, 91, 137, 128)         147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (1, 45, 68, 128)          0         
__________

In [189]:
#Choosing the layer for activation features 
settings = {
    'features': {
        'block3_conv4': 1.5,
        'block4_conv2': 1.5,
    },
    'continuity': 0.1,
}

In [190]:
def continuity_loss(x):
    a = K.square(x[:, :img_height-1, :img_width-1, :] -
                 x[:, 1:, :img_width-1, :])
    b = K.square(x[:, :img_height-1, :img_width-1, :] -
                 x[:, :img_height-1, 1:, :])
    return K.sum(K.pow(a+b, 1.25))

In [191]:
K._LEARNING_PHASE = tf.constant(0)
loss = K.variable(0)
for layer_name in settings['features']:
    coeff = settings['features'][layer_name]
    layer_output = layer_dict[layer_name].output
    shape= layer_output.shape
    scaling = int(np.prod(shape))
    loss+= coeff*K.sum(K.square(layer_output[:, 2:-2, 2:-2, :]))/scaling

In [192]:
loss += settings['continuity'] * continuity_loss(dream) / (img_height*img_width*3)

In [193]:
grads = K.gradients(loss, dream)[0]
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
collecting_grad_and_loss = K.function([dream], [loss, grads])

In [194]:
def loss_and_grads(x):
        outs = collecting_grad_and_loss(x)
        loss_value = outs[0]
        if len(outs[1:]) == 1:
            grad_values = outs[1].flatten().astype('float64')
        else:
            grad_values = np.array(outs[1:]).flatten().astype('float64')
        return loss_value, grad_values

In [195]:
# Gradient ascent
def gradient_ascent(x, iterations, step):
    for i in range(iterations):
        outs = loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1]
        if loss_value <=0:
            break
        print 'Loss value at', i, ':', loss_value
        x += step * grad_values.reshape((1,img_height,img_width,3))
    return x

In [196]:
result = gradient_ascent(x,iterations=20,step=0.1)

Loss value at 0 : 1.58512e+06
Loss value at 1 : 1.61896e+06
Loss value at 2 : 1.64742e+06
Loss value at 3 : 1.6728e+06
Loss value at 4 : 1.69856e+06
Loss value at 5 : 1.7237e+06
Loss value at 6 : 1.74857e+06
Loss value at 7 : 1.77369e+06
Loss value at 8 : 1.79846e+06
Loss value at 9 : 1.82366e+06
Loss value at 10 : 1.84882e+06
Loss value at 11 : 1.87439e+06
Loss value at 12 : 1.89995e+06
Loss value at 13 : 1.92585e+06
Loss value at 14 : 1.95164e+06
Loss value at 15 : 1.97769e+06
Loss value at 16 : 2.00371e+06
Loss value at 17 : 2.0298e+06
Loss value at 18 : 2.05601e+06
Loss value at 19 : 2.08266e+06


In [197]:
y= result.reshape((img_height, img_width, 3))
y.shape

(183, 275, 3)

In [198]:
y = y- y.mean()
y /= (y.std() + 1e-5)
y *= 0.1

# clip to [0, 1]
y += 0.5
y = np.clip(y, 0, 1)

y *= 255
# convert BGR to RGB
y = y[:, :, ::-1]
y = np.clip(y, 0, 255).astype('uint8')

In [199]:
img2= array_to_img(y)

In [200]:
img2.show()